In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sequence_length = 30
n_steps = sequence_length
input_dim = 3  


# # Generate random synthetic data
# synthetic_data = np.random.rand(num_samples, sequence_length, input_dim)

# # Split the data into inputs and outputs
# X = synthetic_data[:, :-1]  # Input data (sequence_length - 1 time steps)
# y = synthetic_data[:, -1]   # Output data (1 time step ahead)

dataset = pd.read_csv("data/train_newdata.csv", delimiter=',')
# print(dataset.head)

dataset_selected = dataset[["node1", "node2", "uwb_range", "tb_node1_yaw", "tb_node2_yaw",  "error"]] #"mocap_range",

uwb_pair = [(1,4), (2,4), (3,4),(4,5),(1,2), (1,3), (1,5), (2,3), (2,4), (2,5), (3,5)]
uwb_data = []
for up in uwb_pair:
    rslt_df = dataset_selected.loc[dataset_selected['node1'] == up[0]]
    rslt_df_new = rslt_df.loc[rslt_df['node2'] == up[1]]
    uwb_data.append(rslt_df_new[["uwb_range", "tb_node1_yaw", "tb_node2_yaw",  "error"]].iloc[:,:].values)

print(uwb_data[1].shape)

X_train = []
y_train = []
for inx, dataset_train in enumerate(uwb_data):
    x_temp = []
    y_temp = []
    for i in range(n_steps,dataset_train.shape[0]):
        x_temp.append(dataset_train[i-n_steps:i,0:3])
        y_temp.append(dataset_train[i,3:])
    # print(np.array(x_temp).shape, np.array(y_temp).shape)
    X_train.append(np.array(x_temp))
    y_train.append(np.array(y_temp))
X_train, y_train = np.array(X_train), np.array(y_train)

2023-09-04 13:40:33.160489: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


(779, 4)


/tmp/ipykernel_33393/2704476149.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train, y_train = np.array(X_train), np.array(y_train)


In [ ]:
# Define the Multi-Scale Multi-Head Attention model
class MultiScaleMultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, num_heads, d_model, num_scales, name="multi_scale_multi_head_attention"):
        super(MultiScaleMultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model
        self.num_scales = num_scales
        self.head_size = d_model // num_heads

        assert self.head_size * num_heads == d_model, "d_model must be divisible by num_heads"

        # Define multi-head attention layers for each scale
        self.attention_layers = [tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=self.head_size,
            value_dim=self.head_size,
        ) for _ in range(num_scales)]

    def call(self, inputs):
        # Apply multi-head attention for each scale
        attention_outputs = []
        for i in range(self.num_scales):
            attention_output = self.attention_layers[i](inputs, inputs)
            attention_outputs.append(attention_output)

        # Concatenate attention outputs from different scales
        concatenated_attention = tf.concat(attention_outputs, axis=-1)

        return concatenated_attention


In [ ]:
lstm_models = []
for inx in range(X_train.shape[0]):
    # Create a simple forecasting model using Multi-Scale Multi-Head Attention
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(sequence_length, input_dim)),
        MultiScaleMultiHeadAttention(num_heads=4, d_model=32, num_scales=3),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1)  # Output layer for forecasting
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='mse')
    lstm_models.append(model)

In [ ]:
history_list = []
earlyStopCallBack = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=8)
for inx, lm in enumerate(lstm_models):
    print(f"------------ In data pair {inx} --------------")
    history = lm.fit(X_train[inx], y_train[inx], epochs=5, batch_size=16, validation_split=0.3, callbacks=[earlyStopCallBack] )
    history_list.append(history)
    
print(f"------------ Training Ended --------------")


In [ ]:
for history in history_list:
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model mean_squared_error')
    plt.ylabel('mean_squared_error')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
dataset = pd.read_csv("data/test_newdata.csv", delimiter=',')
# print(dataset.head)

uwb_pair = [(1,4), (2,4), (3,4),(4,5),(1,2), (1,3), (1,5), (2,3), (2,4), (2,5), (3,5)]
uwb_data_test = []
for up in uwb_pair:
    rslt_df = dataset.loc[dataset['node1'] == up[0]]
    rslt_df_new = rslt_df.loc[rslt_df['node2'] == up[1]]
    uwb_data_test.append(rslt_df_new[["uwb_range", "tb_node1_yaw", "tb_node2_yaw",  "error"]].iloc[:200,:].values)

print(uwb_data_test[1].shape)


X_test = []
y_test = []
for inx, dataset_test in enumerate(uwb_data_test):
    x_temp = []
    y_temp = []
    for i in range(n_steps,dataset_test.shape[0]):
        x_temp.append(dataset_test[i-n_steps:i,0:3])
        y_temp.append(dataset_test[i,3:])
    print(np.array(x_temp).shape, np.array(y_temp).shape)
    X_test.append(np.array(x_temp))
    y_test.append(np.array(y_temp))
X_test, y_test = np.array(X_test), np.array(y_test)

print(X_test[1].shape, y_test[1].shape)

Predicted_X = []
for inx in range(len(lstm_models)):
    predict_x = lstm_models[inx].predict(X_test[inx], verbose=0)
    Predicted_X.append(predict_x)
    mse = tf.keras.losses.MeanSquaredError()
    print(mse(y_test[inx], predict_x).numpy())


for inx in range(len(lstm_models)):
    lstm_models[inx].save('models/lstm_uwb_{}'.format(inx))

In [ ]:
for p in range(len(Predicted_X)):
    plt.plot(y_test[p], color='red', label= 'Real UWB Error')
    plt.plot(Predicted_X[p], color='blue', label='Estimated UWB Error')
    plt.title("UWB Error Estimation")
    plt.xlabel('Time')
    plt.ylabel('UWB Error')
    plt.legend()
    plt.show()